In [1]:
import os
os.environ["OPENAI_API_KEY"] ="paste your api key"

In [2]:
!pip install -q langchain-openai langchain-community langchain-core requests duckduckgo-search

In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
import requests

In [4]:
from langchain_community.tools import DuckDuckGoSearchRun

search_tool = DuckDuckGoSearchRun()

In [5]:
@tool
def get_weather_data(city: str) -> str:
  """
  This function fetches the current weather data for a given city
  """
  url = f'https://api.weatherstack.com/current?access_key=4d1d8ae207a8c845a52df8a67bf3623e&query={city}'

  response = requests.get(url)

  return response.json()

In [6]:
llm = ChatOpenAI()

In [7]:
from langchain.agents import create_react_agent, AgentExecutor
from langchain import hub

In [8]:
# Step 2: Pull the ReAct prompt from LangChain Hub
prompt = hub.pull("hwchase17/react")  # pulls the standard ReAct agent prompt


In [9]:
# Step 3: Create the ReAct agent manually with the pulled prompt
agent = create_react_agent(
    llm=llm,
    tools=[search_tool, get_weather_data],
    prompt=prompt
)

In [10]:
# Step 4: Wrap it with AgentExecutor
agent_executor = AgentExecutor(
    agent=agent,
    tools=[search_tool, get_weather_data],
    verbose=True
)

In [11]:
# Step 5: Invoke
response = agent_executor.invoke({"input": "Find the capital of jharkhand, then find it's current weather condition"})
print(response)



> Entering new AgentExecutor chain...
We need to first find out the capital of Jharkhand, then check the current weather there.
Action: duckduckgo_search
Action Input: "Capital of Jharkhand"18 Dec 2024 — The city of Ranchi is its capital and Dumka its sub-capital. The state is known for its waterfalls, hills and holy places; Baidyanath Dham, Parasnath, Dewri and ... 3 weeks ago - Ranchi (Hindi: rāñcī, pronounced [ˈɾaːɲt͡ɕiː]) is the capital city and also the largest district by population of the Indian state of Jharkhand. Ranchi was the centre of the Jharkhand movement, which called for a separate state for the tribal regions of South Bihar, northern Odisha, ... 3 weeks ago - Dhanbad is the second-most populated city in the Indian state of Jharkhand after Jamshedpur and a major financial hub of Jharkhand. In terms of economy, Dhanbad has the largest economy in the state of Jharkhand and it is often referred to as the 'Economic Capital of Jharkhand'. Dhanbad is also ... August 3, 2025

In [12]:
response['output']

"The capital of Jharkhand is Ranchi, but we couldn't retrieve the current weather data for Ranchi due to reaching the usage limit."